In [1]:
!ls ./datasets/STRATIFIED_NWPU-RESISC45-500-TRAINVAL7015

train  val


In [2]:
!nvidia-smi

Tue Jun  3 21:45:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.86.16              Driver Version: 572.16         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   52C    P5              8W /   85W |     649MiB /   8188MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import tensorflow as tf

import keras
import keras_hub
import joblib
import os

2025-06-03 21:45:08.379853: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-03 21:45:08.563447: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748961908.636767    1617 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748961908.659357    1617 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-03 21:45:08.822865: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
tf.config.list_logical_devices()

I0000 00:00:1748961913.700008    1617 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

# Data Preparation

In [5]:
path_ds = "./datasets/STRATIFIED_NWPU-RESISC45-500-TRAINVAL7015"

In [6]:
img_size = (224, 224)
batch_size = 10

# TRAIN DATA
print("================== TRAIN DATA")
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    directory = f"{path_ds}/train",
    shuffle = True,
    image_size = img_size,
    batch_size = batch_size
)
print(f"Len data within each batch: {len(train_data)}")


# VAL DATA
print("================== VAL DATA")
val_data = tf.keras.preprocessing.image_dataset_from_directory(
    directory = f"{path_ds}/val",
    shuffle = True,
    image_size = img_size,
    batch_size = batch_size
)
print(f"Len data within each batch: {len(val_data)}")

================== TRAIN DATA
Found 15750 files belonging to 45 classes.
Len data within each batch: 1575
================== VAL DATA
Found 3375 files belonging to 45 classes.
Len data within each batch: 338


In [7]:
classes = train_data.class_names
classes

['airplane',
 'airport',
 'baseball_diamond',
 'basketball_court',
 'beach',
 'bridge',
 'chaparral',
 'church',
 'circular_farmland',
 'cloud',
 'commercial_area',
 'dense_residential',
 'desert',
 'forest',
 'freeway',
 'golf_course',
 'ground_track_field',
 'harbor',
 'industrial_area',
 'intersection',
 'island',
 'lake',
 'meadow',
 'medium_residential',
 'mobile_home_park',
 'mountain',
 'overpass',
 'palace',
 'parking_lot',
 'railway',
 'railway_station',
 'rectangular_farmland',
 'river',
 'roundabout',
 'runway',
 'sea_ice',
 'ship',
 'snowberg',
 'sparse_residential',
 'stadium',
 'storage_tank',
 'tennis_court',
 'terrace',
 'thermal_power_station',
 'wetland']

## Layers for Data Scaling and Augmentation

In [8]:
augmentation_layers = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.RandomZoom(height_factor=0.1, width_factor=0.1),
    tf.keras.layers.RandomFlip("horizontal"),
])

rescaling_layer = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
])


train_data_no_augmentation = train_data.map(lambda x, y: (rescaling_layer(x), y))
val_data = val_data.map(lambda x, y: (rescaling_layer(x), y))
train_data_with_augmentation = train_data.map(lambda x, y: (augmentation_layers(x), y))


print(f"""Length of new data in each batch
train data: {len(train_data_no_augmentation)}
val data: {len(val_data)}
train data with augmentation: {len(train_data_with_augmentation)}
""")

Length of new data in each batch
train data: 1575
val data: 338
train data with augmentation: 1575



## Apply Data Prefetch

In [9]:
AUTOTUNE = tf.data.AUTOTUNE
train_data_no_augmentation = train_data_no_augmentation.prefetch(buffer_size=AUTOTUNE)
val_data = val_data.prefetch(buffer_size=AUTOTUNE)
train_data_with_augmentation = train_data_with_augmentation.prefetch(buffer_size=AUTOTUNE)

# Modelling

In [10]:
img_shape = (batch_size, *img_size, 3)
img_shape, img_shape[1:]

((10, 224, 224, 3), (224, 224, 3))

## Model Saving Callback (Epoch-based)

In [11]:
class EpochModelCheckpoint(tf.keras.callbacks.ModelCheckpoint):
    def __init__(self,
                 filepath,
                 frequency=1,
                 monitor="val_accuracy",
                 verbose=1,
                 save_best_only=False,
                 save_weights_only=False,
                 mode="max",
                 save_freq="epoch",
                 initial_value_threshold=None):

        super(EpochModelCheckpoint, self).__init__(filepath,
                                                   monitor,
                                                   verbose,
                                                   save_best_only,
                                                   save_weights_only,
                                                   mode,
                                                   save_freq,
                                                   initial_value_threshold)
        self.epochs_since_last_save = 0
        self.frequency = frequency

    def on_epoch_end(self, epoch, logs=None):
        self.epochs_since_last_save += 1
        if self.epochs_since_last_save % self.frequency == 0:
            self._save_model(epoch=epoch, batch=None, logs=logs)

    def on_train_batch_end(self, batch, logs=None):
        pass

## Prepare Dir for Model's History

In [12]:
save_history_path = "./outputs/histories"
os.makedirs(save_history_path, exist_ok=True)

## ConvNeXt-Tiny Function Definition

In [ ]:
def convnext_tiny(epoch_size: int, augment: bool=False, frequency=5):
    model_name = f"{'base' if augment == False else 'aug'}_convnext-tiny_{epoch_size}"

    model = tf.keras.applications.ConvNeXtTiny(
        include_top=False,
        include_preprocessing=False,
        weights="imagenet",
        input_shape=img_shape[1:],
        pooling=None,
        name="convnext_tiny",
    )

    model.trainable = True

    inputs = tf.keras.layers.Input(shape=img_shape[1:])
    x = model(inputs)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(128, activation="relu")(x)
    x = tf.keras.layers.Dense(256, activation="relu")(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    outputs = tf.keras.layers.Dense(len(classes))(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name=model_name)

    model.compile(
        optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-5, weight_decay=1e-2),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )

    callback = EpochModelCheckpoint(filepath=f"./outputs/{model_name}/{model_name}_epoch{{epoch:04d}}.keras", frequency=frequency)
    
    model.summary()
    print(f"\n##### Optimizer Func Information\n{model.optimizer.get_config()}")
    print(f"\n##### Loss Func Information\n{model.loss.get_config()}")
    
    return model, callback

## 15 Epochs No Augmentation

In [14]:
epoch_size = 15
base_convnexttiny_15, base_convnexttiny_15_ckpt_callback = convnext_tiny(epoch_size=epoch_size, augment=False)

Model: "base_convnext-tiny_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ convnext_tiny (Functional)      │ (None, 7, 7, 768)      │    27,820,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 768)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        98,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 45)             │        23,085 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,106,253 (107.22 MB)

 Trainable params: 28,106,253 (107.22 MB)

 Non-trainable params: 0 (0.00 B)


##### Optimizer Func Information
{'name': 'adamw', 'learning_rate': 9.999999747378752e-06, 'weight_decay': 0.01, 'clipnorm': None, 'global_clipnorm': None, 'clipvalue': None, 'use_ema': False, 'ema_momentum': 0.99, 'ema_overwrite_frequency': None, 'loss_scale_factor': None, 'gradient_accumulation_steps': None, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}

##### Loss Func Information
{'name': 'sparse_categorical_crossentropy', 'reduction': 'sum_over_batch_size', 'from_logits': True, 'ignore_class': None}


In [15]:
%%time

history_base_convnexttiny_15 = base_convnexttiny_15.fit(
    train_data_no_augmentation,
    validation_data=val_data,
    epochs=epoch_size,
    callbacks=[base_convnexttiny_15_ckpt_callback]
)

Epoch 1/15


I0000 00:00:1748848993.514688   13458 service.cc:148] XLA service 0x778a1c018760 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748848993.516314   13458 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-06-02 14:23:14.236808: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1748848996.453786   13458 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-06-02 14:23:19.805430: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot', 76 bytes spill stores, 76 bytes spill loads

2025-06-02 14:23:19.912833: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot', 116 byt

1575/1575 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.1669 - loss: 3.3868

2025-06-02 14:28:09.181748: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1937', 8 bytes spill stores, 8 bytes spill loads



1575/1575 ━━━━━━━━━━━━━━━━━━━━ 321s 173ms/step - accuracy: 0.1670 - loss: 3.3864 - val_accuracy: 0.6385 - val_loss: 1.4835
Epoch 2/15
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 260s 165ms/step - accuracy: 0.7195 - loss: 1.1823 - val_accuracy: 0.8130 - val_loss: 0.7203
Epoch 3/15
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 263s 167ms/step - accuracy: 0.8458 - loss: 0.5813 - val_accuracy: 0.8628 - val_loss: 0.4954
Epoch 4/15
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 259s 164ms/step - accuracy: 0.9050 - loss: 0.3530 - val_accuracy: 0.8874 - val_loss: 0.3885
Epoch 5/15
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.9401 - loss: 0.2304
Epoch 5: saving model to ./outputs/base_convnext-tiny_15/base_convnext-tiny_15_epoch0005.keras
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 266s 169ms/step - accuracy: 0.9401 - loss: 0.2304 - val_accuracy: 0.8942 - val_loss: 0.3529
Epoch 6/15
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 260s 165ms/step - accuracy: 0.9622 - loss: 0.1491 - val_accuracy: 0.9034 - val_loss: 0.3146
Epoch 7/15
1575/1575 ━━━━━━━━━━━━

In [16]:
joblib.dump(history_base_convnexttiny_15.history, os.path.join(save_history_path, "history_base_convnext-tiny_15.joblib"))

['./outputs/histories/history_base_convnext-tiny_15.joblib']

## 15 Epochs with Augmentation

In [14]:
epoch_size = 15
aug_convnexttiny_15, aug_convnexttiny_15_ckpt_callback = convnext_tiny(epoch_size=epoch_size, augment=True)

Model: "aug_convnext-tiny_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ convnext_tiny (Functional)      │ (None, 7, 7, 768)      │    27,820,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 768)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        98,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 45)             │        23,085 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,106,253 (107.22 MB)

 Trainable params: 28,106,253 (107.22 MB)

 Non-trainable params: 0 (0.00 B)


##### Optimizer Func Information
{'name': 'adamw', 'learning_rate': 9.999999747378752e-06, 'weight_decay': 0.01, 'clipnorm': None, 'global_clipnorm': None, 'clipvalue': None, 'use_ema': False, 'ema_momentum': 0.99, 'ema_overwrite_frequency': None, 'loss_scale_factor': None, 'gradient_accumulation_steps': None, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}

##### Loss Func Information
{'name': 'sparse_categorical_crossentropy', 'reduction': 'sum_over_batch_size', 'from_logits': True, 'ignore_class': None}


In [15]:
%%time

history_aug_convnexttiny_15 = aug_convnexttiny_15.fit(
    train_data_with_augmentation,
    validation_data=val_data,
    epochs=epoch_size,
    callbacks=[aug_convnexttiny_15_ckpt_callback] 
)

Epoch 1/15


I0000 00:00:1748853775.697488   22943 service.cc:148] XLA service 0x73b358013c20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748853775.699114   22943 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-06-02 15:42:56.255467: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1748853778.363266   22943 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-06-02 15:43:00.528881: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot', 116 bytes spill stores, 116 bytes spill loads

2025-06-02 15:43:00.635644: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot', 76 by

1575/1575 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step - accuracy: 0.1564 - loss: 3.4628

2025-06-02 15:47:49.009095: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1937', 8 bytes spill stores, 8 bytes spill loads



1575/1575 ━━━━━━━━━━━━━━━━━━━━ 324s 178ms/step - accuracy: 0.1565 - loss: 3.4624 - val_accuracy: 0.6412 - val_loss: 1.4794
Epoch 2/15
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 264s 167ms/step - accuracy: 0.7124 - loss: 1.1934 - val_accuracy: 0.8018 - val_loss: 0.7173
Epoch 3/15
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 265s 168ms/step - accuracy: 0.8357 - loss: 0.6151 - val_accuracy: 0.8604 - val_loss: 0.4949
Epoch 4/15
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 263s 167ms/step - accuracy: 0.8817 - loss: 0.4079 - val_accuracy: 0.8827 - val_loss: 0.3831
Epoch 5/15
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 0.9171 - loss: 0.2890
Epoch 5: saving model to ./outputs/aug_convnext-tiny_15/aug_convnext-tiny_15_epoch0005.keras
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 267s 169ms/step - accuracy: 0.9172 - loss: 0.2890 - val_accuracy: 0.8981 - val_loss: 0.3356
Epoch 6/15
1575/1575 ━━━━━━━━━━━━━━━━━━━━ 264s 167ms/step - accuracy: 0.9413 - loss: 0.2137 - val_accuracy: 0.9087 - val_loss: 0.2941
Epoch 7/15
1575/1575 ━━━━━━━━━━━━━━

In [16]:
joblib.dump(history_aug_convnexttiny_15.history, os.path.join(save_history_path, "history_aug_convnext-tiny_15.joblib"))

['./outputs/histories/history_aug_convnext-tiny_15.joblib']